This code build embeddings for customer and article from text description embeddings. Customer embedding are averaged across the dimension of the embeddings.


In [1]:
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import pickle
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

path = '../input/h-and-m-personalized-fashion-recommendations/articles.csv'
 
df = pd.read_csv(path, dtype={'article_id':str}).astype(str)

In [2]:
#@title Load the Universal Sentence Encoder's TF Hub module

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)

2022-05-18 15:38:05.178698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 15:38:05.323762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 15:38:05.324790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 15:38:05.326146: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [3]:
desc = df['detail_desc'].unique()

In [4]:
%%time
embeds = model(desc)

CPU times: user 7.05 s, sys: 989 ms, total: 8.04 s
Wall time: 8.05 s


In [5]:
desc_embedding_pairs = {i:embed.numpy() for i, embed in zip(desc, embeds)}

In [6]:
df = df[['article_id', 'detail_desc']]

In [7]:
df['embeddings'] = df.detail_desc.map(desc_embedding_pairs)

In [8]:
df = df[['article_id', 'embeddings']]

In [9]:
f = open('article_embeddings_from_text.pickle', 'wb')
pickle.dump(df.values, f)

In [10]:
trans = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
                    usecols=['customer_id', 'article_id'],
                   dtype=str)

In [11]:
trans = trans.merge(df, on='article_id').drop('article_id', axis=1)

In [12]:
%%time
customer_embeddings = trans.groupby('customer_id').mean()

CPU times: user 4min 25s, sys: 4.45 s, total: 4min 29s
Wall time: 4min 30s


In [13]:
f = open('customer_embeddings_from_text.pickle', 'wb')

In [14]:
pickle.dump(customer_embeddings, f)